In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from path import Path as p

In [12]:
PATH = "../../../data/"
PATH2 = "../../../data/Flicker8k_Dataset/"
sz=224
bs = 64
n = 1000 # Number of dogs and cats to train model on
# imbalanced classes
CATDOG = 0 # Dummy class variables
NOTCATDOG = 1 # Not Cat or Not Dog

In [4]:
def get_names(filelist, suffix):
    return [suffix + f.name for f in filelist]

In [13]:
train_cats = get_names(p(PATH + "dogscats/train/cats/").files(), "dogscats/train/cats/")
train_dogs = get_names(p(PATH + "dogscats/train/dogs/").files(), "dogscats/train/dogs/")
valid_cats = get_names(p(PATH + "dogscats/valid/cats/").files(), "dogscats/valid/cats/")
valid_dogs = get_names(p(PATH + "dogscats/valid/dogs/").files(), "dogscats/valid/dogs/")
flickr = get_names(p(PATH2).files(), "Flicker8k_Dataset/")
print(len(train_cats))
print(len(train_dogs))
print(len(valid_cats))
print(len(valid_dogs))
print(len(flickr))

11500
11500
1000
1000
8091


In [14]:
catdogs = list(np.random.choice(train_cats, n, False)) + list(valid_cats)
notcatdogs = list(np.random.choice(flickr, n + 1000, False))
fnames = catdogs + notcatdogs
y = np.array([CATDOG]*len(catdogs) + [NOTCATDOG]*len(notcatdogs))
classes = list(set(y))
v_cat_dog_idx = range(n, n + len(valid_cats))
v_not_idx = range(len(catdogs) + n, len(fnames))
val_idxs = list(v_cat_dog_idx) + list(v_not_idx)

# fnames: file names
# y: numpy array which contains target labels ordered by filenames.
# classes: a list of all labels/classifications, [0, 1]
# val_idxs: index of images to be used for validation.

In [15]:
# Let's set up our model using the pretrained Resnet34 Imagenet model
arch=resnet34
data = ImageClassifierData.from_names_and_array(PATH, fnames, y, classes, \
                                                val_idxs, bs=bs, \
                                                tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:22<00:00,  1.42it/s]


In [16]:
# Use a learning rate of 0.01 and train for 5 epochs
lr = 0.01
epochs = 5
learn.fit(lr, epochs)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.126184   0.034782   0.99      
    1      0.068991   0.023006   0.993                                                                                 
    2      0.045593   0.019717   0.9945                                                                                
    3      0.028688   0.022593   0.993                                                                                 
    4      0.022849   0.018565   0.9945                                                                                



[array([0.01856]), 0.9945]